<a href="https://colab.research.google.com/github/Wozny614/mispy/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
full = train.append(test, ignore_index=True)


In [4]:
full.describe()     # the information of only numbers
full.info()         # the information of all variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [6]:
full['Age'] = full['Age'].fillna(full['Age'].mean())
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())
full['Embarked'] = full['Embarked'].fillna('S')
full['Cabin'] = full['Cabin'].fillna('U')       # "Unknown"

In [7]:
sex_mapDict = {'male': 1, 'female': 0}
full['Sex'] = full['Sex'].map(sex_mapDict)

In [8]:
embarkedDF = pd.get_dummies(full['Embarked'], prefix='Embarked')
embarkedDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, embarkedDF, on='PassengerId')

In [9]:


PclassDF = pd.get_dummies(full['Pclass'], prefix='Pclass')
PclassDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, PclassDF, on='PassengerId')



In [10]:
def getTitle(name):
    str1 = name.split(',')[1]
    str2 = str1.split('.')[0]
    str3 = str2.strip()
    return str3

titleDF = pd.DataFrame()
titleDF['Title'] = full['Name'].map(getTitle)
title_mapDict = {"Capt":         "Officer",
                 "Col":          "Officer",
                 "Major":        "Officer",
                 "Jonkheer":     "Royalty",
                 "Don":          "Royalty",
                 "Sir":          "Royalty",
                 "Dr":           "Officer",
                 "Rev":          "Officer",
                 "the Countess": "Royalty",
                 "Dona":         "Royalty",
                 "Mme":          "Mrs",
                 "Mlle":         "Miss",
                 "Ms":           "Mrs",
                 "Mr":           "Mr",
                 "Mrs":          "Mrs",
                 "Miss":         "Miss",
                 "Master":       "Master",
                 "Lady":         "Royalty"}
titleDF['Title'] = titleDF['Title'].map(title_mapDict)
titleDF = pd.get_dummies(titleDF['Title'])
titleDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, titleDF, on='PassengerId')

In [11]:
cabinDF = pd.DataFrame()
cabinDF['Cabin'] = full['Cabin'].map(lambda c: c[0])
cabinDF = pd.get_dummies(cabinDF['Cabin'], prefix='Cabin')
cabinDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, cabinDF, on='PassengerId')

In [12]:
familyDF = pd.DataFrame()
familyDF['FamilySize'] = full['Parch'] + full['SibSp'] + 1
familyDF['Family_Single'] = familyDF['FamilySize'].map(lambda s: 1 if (s == 1) else 0)
familyDF['Family_Small'] = familyDF['FamilySize'].map(lambda s: 1 if (2 <= s <= 4) else 0)
familyDF['Family_Large'] = familyDF['FamilySize'].map(lambda s: 1 if (s >= 5) else 0)
familyDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, familyDF, on='PassengerId')

In [13]:
corrDF = full.corr()
print(corrDF['Survived'].sort_values(ascending=False))
full_X = full[['Mrs', 'Miss', 'Pclass_1', 'Family_Small', 'Fare', 'Family_Single', 'Pclass_3', 'Sex', 'Mr']]

Survived         1.000000
Mrs              0.344935
Miss             0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Cabin_B          0.175095
Embarked_C       0.168240
Cabin_D          0.150716
Cabin_E          0.145321
Cabin_C          0.114652
Pclass_2         0.093349
Master           0.085221
Parch            0.081629
Cabin_F          0.057935
Royalty          0.033391
Cabin_A          0.022287
FamilySize       0.016639
Cabin_G          0.016040
Embarked_Q       0.003650
PassengerId     -0.005007
Cabin_T         -0.026456
Officer         -0.031316
SibSp           -0.035322
Age             -0.070323
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Cabin_U         -0.316912
Pclass_3        -0.322308
Pclass          -0.338481
Sex             -0.543351
Mr              -0.549199
Name: Survived, dtype: float64


In [14]:
sourceRow = full['Survived'].count()
source_X = full_X.loc[0:sourceRow-1, :]
source_y = full.loc[0:sourceRow-1, 'Survived']
predict_X = full_X.loc[sourceRow:, :]

In [15]:
train_X, test_X, train_y, test_y = train_test_split(source_X, source_y, train_size=.8)
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(train_X, train_y)
print(model.score(test_X, test_y))

0.7932960893854749


In [18]:
predict_y = model.predict(predict_X).astype(int)
passenger_Id = full.loc[sourceRow:, 'PassengerId']
predictDF = pd.DataFrame({'PassengerId': passenger_Id,
                          'Survived': predict_y})
predictDF.to_csv('prediction1.csv', index=False)

In [21]:
pd.read_csv('/content/prediction1.csv')

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
